### IMPORTING REQUIRED LIBRARIES

In [104]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
import smtplib

In [105]:
# 1st step for this project is to extract the data from the Amazon website:


URL = 'https://www.amazon.in/DJI-Vlogging-Stabilization-Tracking-Photography/dp/B0CG19FGQ5?ref_=Oct_d_obs_d_1389174031_0&pd_rd_w=2wSOo&content-id=amzn1.sym.a0183515-a55a-48ac-a863-406c0a598721&pf_rd_p=a0183515-a55a-48ac-a863-406c0a598721&pf_rd_r=NCK4FFH9JYRH6FN7FYBB&pd_rd_wg=wpy45&pd_rd_r=783e2e1f-a51a-4cbe-b0bd-296685d61daa&pd_rd_i=B0CG19FGQ5'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

# requesting the website:
page = requests.get(URL, headers = headers)

# to parse the HTML text:
soup1 = BeautifulSoup(page.content, "html.parser")

# to prettify the text
soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

# to extract the title of the product:
title = soup2.find(id = 'productTitle').get_text().strip()

# to extract the price of the product:
#price = soup2.find(class_ = 'a-price-whole').get_text().strip()

# Extract and clean price components
price_whole = soup2.find('span', class_='a-price-whole').get_text()
price_whole_clean = ''.join(c for c in price_whole if c.isdigit())  # Removes commas, newlines, dots, spaces

price_fraction = soup2.find('span', class_='a-price-fraction').get_text().strip()

# Combine and convert
price = float(f"{price_whole_clean}.{price_fraction}")



print(title)
print(price)

# to get only the numeric value for the price:
# price = price[1:]

DJI Osmo Pocket 3 Creator Combo, Vlogging Camera with 1'' CMOS & 4K/120fps Video, 3-Axis Stabilization, Face/Object Tracking, Fast Focusing, Mic Included for Clear Sound, Small Camera for Photography
69990.0


In [106]:
# Then, we create a timespamp for our output to track when data was collected in this system:
import datetime

today = datetime.date.today()
print(today)

2025-06-24


In [107]:
# Now we create a CSV file and write headers and data into the file:
import csv

header = ['Product Title', 'Price', 'Date']
data = [title, price, today]

# Basic File handling: w -> to write, '' -> each row is without any spaces
with open ('Web_Scraper_data.csv', 'w', newline = '', encoding = 'UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerow(data)
    

In [108]:
# Reading our new data file:
import pandas as pd

df = pd.read_csv('/home/aayan/Portfolio_Python/Amazon_Web_Scraper/Web_Scraper_data.csv')
print(df)

                                       Product Title    Price        Date
0  DJI Osmo Pocket 3 Creator Combo, Vlogging Came...  69990.0  2025-06-24


In [109]:
# Now, we append data to this CSV file:
# a+ -> append symbol

with open ('Web_Scraper_data.csv', 'a+', newline = '', encoding = 'UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(data)

In [110]:
# Finally, we combine all of the above code into one function:

def check_price():
    URL = 'https://www.amazon.in/DJI-Vlogging-Stabilization-Tracking-Photography/dp/B0CG19FGQ5?ref_=Oct_d_obs_d_1389174031_0&pd_rd_w=2wSOo&content-id=amzn1.sym.a0183515-a55a-48ac-a863-406c0a598721&pf_rd_p=a0183515-a55a-48ac-a863-406c0a598721&pf_rd_r=NCK4FFH9JYRH6FN7FYBB&pd_rd_wg=wpy45&pd_rd_r=783e2e1f-a51a-4cbe-b0bd-296685d61daa&pd_rd_i=B0CG19FGQ5'
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    page = requests.get(URL, headers = headers)
    soup1 = BeautifulSoup(page.content, "html.parser")
    soup2 = BeautifulSoup(soup1.prettify(), "html.parser")
    title = soup2.find(id = 'productTitle').get_text().strip()
    
    # Extract and clean price components
    price_whole = soup2.find('span', class_='a-price-whole').get_text()
    price_whole_clean = ''.join(c for c in price_whole if c.isdigit())  # Removes commas, newlines, dots, spaces
    price_fraction = soup2.find('span', class_='a-price-fraction').get_text().strip()
    # Combine and convert
    price = float(f"{price_whole_clean}.{price_fraction}")



    today = datetime.date.today()

    header = ['Product Title', 'Price', 'Date']
    data = [title, price, today]

    with open ('Web_Scraper_data.csv', 'a+', newline = '', encoding = 'UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(data)

    if(price < 65000):
        send_mail()
        



### START/STOP THE FUNCTION:

In [111]:
# Running the above function after a set time (a day) and inputting data into our CSV file:

while(True):
    check_price()
    time.sleep(86400)

KeyboardInterrupt: 

### CHECKING THE PRICE DATA OVER TIME:

In [ ]:
import pandas as pd

df = pd.read_csv('/home/aayan/Portfolio_Python/Amazon_Web_Scraper/Web_Scraper_data.csv')

print(df) # to check periodically

### SENDING AN EMAIL

In [ ]:
# Sending an email when a price hits below a threshold level, which we can introduce:

def send_mail():
    server = smtplib.SMTP_SSL('smtp.gmail.com',465)
    server.ehlo()
    #server.starttls()
    sender_email = 'aayanak04@gmail.com'
    receiver_email = 'aayanak04@gmail.com'
    password = 'xxxxxxxxxxxxxx'
    server.login(sender_email, password) # Entering my password, which is not shared here
    
    subject = "The camera you want is below ₹65,000! Now is your chance to buy!"
    body = "Aayan, This is the moment you have been waiting for! This is your chance to pick up the action camera of your dreams. Don't mess it up! 👉 Here is your link: https://www.amazon.in/DJI-Vlogging-Stabilization-Tracking-Photography/dp/B0CG19FGQ5?ref_=Oct_d_obs_d_1389174031_0&pd_rd_w=2wSOo&content-id=amzn1.sym.a0183515-a55a-48ac-a863-406c0a598721&pf_rd_p=a0183515-a55a-48ac-a863-406c0a598721&pf_rd_r=NCK4FFH9JYRH6FN7FYBB&pd_rd_wg=wpy45&pd_rd_r=783e2e1f-a51a-4cbe-b0bd-296685d61daa&pd_rd_i=B0CG19FGQ5"
   
    msg = f"Subject: {subject}\n\n{body}"
    
    server.sendmail(sender_email, receiver_email, msg)
    server.quit()
    print("✅ Email sent successfully!")
     
    